In [3]:
#!/usr/bin/env python3
import csv
import matplotlib
matplotlib.use('Agg')  # Forza l'uso di un backend non interattivo per salvare file
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
import os
import sys # Importa sys per usare sys.exit

# =============================================================================
# Script Python per plottare Coefficienti di Forza (Cd, Cl) e y+ da OpenFOAM
# - Trova automaticamente il percorso del caso dalla directory corrente.
# - Cerca i file di dati per 'forceCoeffs' e 'yPlus' in postProcessing.
# - Gestisce il formato multi-patch del file di output di yPlus.
# - Crea DUE GRAFICI SEPARATI:
#     1. Grafico_Forze.png (con Cd e Cl)
#     2. Grafico_yPlus_[patch].png (con y+ min/avg/max, se dati trovati)
# - Salva i grafici PNG nella directory principale del caso.
# =============================================================================

print("--- Inizio Script Plotting OpenFOAM ---")

# --- Fase 1: Trovare i percorsi e i file di dati ---

# Usa la directory corrente come percorso del caso
case_path = os.getcwd()
print(f"Percorso del caso (directory corrente): {case_path}")

# --- 1a: Dati Force Coefficients ---
force_func_dir = "forceCoeffs1" # Assicurati che sia il nome corretto della tua functionObject directory
force_path = os.path.join(case_path, "postProcessing", force_func_dir)
force_data_file = None
possible_force_files = ["coefficient.dat", "forceCoeffs.dat"] # Aggiungi altri se necessario

print(f"Ricerca directory risultati forze in: {force_path}")
if not os.path.isdir(force_path):
    print(f"ERRORE: Directory {force_path} non trovata.")
    # Potresti voler continuare senza forze se vuoi solo y+
    # sys.exit(1) # Commentato per permettere di plottare solo y+ se le forze mancano
    force_data_found = False
else:
    force_data_found = False
    # Cerca file dati forze nelle sottodirectory temporali
    time_dirs = sorted([d for d in os.listdir(force_path) if os.path.isdir(os.path.join(force_path, d)) and d.replace('.', '', 1).isdigit()], key=float, reverse=True)
    if time_dirs:
        latest_time_dir = os.path.join(force_path, time_dirs[0])
        for fname in possible_force_files:
            fpath = os.path.join(latest_time_dir, fname)
            if os.path.exists(fpath):
                force_data_file = fpath
                force_data_found = True
                print(f"Trovato file dati forze in directory temporale: {force_data_file}")
                break
    # Fallback: cerca direttamente nella directory della funzione
    if not force_data_found:
        for fname in possible_force_files:
            fpath = os.path.join(force_path, fname)
            if os.path.exists(fpath):
                force_data_file = fpath
                force_data_found = True
                print(f"Trovato file dati forze direttamente in: {force_data_file}")
                break
    if not force_data_found:
         print(f"Info: Non trovato file dati forze in {force_path} o sue subdir note.")


# --- 1b: Dati yPlus ---
yplus_func_dir = "yPlus1" # Assicurati che sia il nome corretto
yplus_path = os.path.join(case_path, "postProcessing", yplus_func_dir)
yplus_data_file = None
possible_yp_files = ["yPlus.dat"] # Il nome è tipicamente questo
yplus_data_found = False # Flag per file y+ trovato
target_patch = "bodyWall" # Patch di interesse per y+ - CAMBIA SE NECESSARIO

print(f"Ricerca directory risultati y+ in: {yplus_path}")
if not os.path.isdir(yplus_path):
    print(f"Info: Directory {yplus_path} non trovata. Grafico y+ non generato.")
else:
    # Cerca file dati y+ nelle sottodirectory temporali (più comune per yPlus.dat)
    time_dirs_yp = sorted([d for d in os.listdir(yplus_path) if os.path.isdir(os.path.join(yplus_path, d)) and d.replace('.', '', 1).isdigit()], key=float, reverse=True)
    if time_dirs_yp:
        latest_time_dir_yp = os.path.join(yplus_path, time_dirs_yp[0])
        for fname in possible_yp_files:
            fpath = os.path.join(latest_time_dir_yp, fname)
            if os.path.exists(fpath):
                yplus_data_file = fpath
                yplus_data_found = True
                print(f"Trovato file dati y+ in directory temporale: {yplus_data_file}")
                break
    # Fallback: cerca direttamente nella directory della funzione (meno comune per yPlus.dat)
    if not yplus_data_found:
         for fname in possible_yp_files:
            fpath = os.path.join(yplus_path, fname)
            if os.path.exists(fpath):
                yplus_data_file = fpath
                yplus_data_found = True
                print(f"Trovato file dati y+ direttamente in: {yplus_data_file}")
                break
    if not yplus_data_found:
        print(f"Info: Non trovato file dati y+ valido in {yplus_path} o sue subdir note. Grafico y+ non generato.")


# --- Fase 2: Lettura dei Dati ---

# Liste per i dati delle forze
colonna_tempo_f = []; colonna_cd = []; colonna_cl = []
force_data_read_success = False

# Liste per i dati di y+
colonna_tempo_yp = []; colonna_yp_min = []; colonna_yp_avg = []; colonna_yp_max = []
yplus_data_read_success = False # Flag per dati y+ letti con successo


# --- 2a: Lettura Dati Forze (se il file è stato trovato) ---
if force_data_found:
    print(f"Leggendo dati forze da: {force_data_file}")
    try:
        with open(force_data_file, 'r') as f_force:
            righe_valide_f = [line for line in f_force if not line.strip().startswith('#')]
            # Tenta di determinare il delimitatore (spazio o tab)
            delimiter_f = '\t' # Default a tab
            if righe_valide_f and '\t' not in righe_valide_f[0] and ' ' in righe_valide_f[0]:
                 delimiter_f = ' ' # Passa a spazio se non ci sono tab ma ci sono spazi
                 print("Info: Rilevato delimitatore spazio per il file forze.")

            # Usa uno split più robusto se il delimitatore è spazio
            if delimiter_f == ' ':
                 for riga in righe_valide_f:
                      riga_pulita = riga.split() # Divide sugli spazi
                      if len(riga_pulita) >= 4: # Assicurati ci siano abbastanza colonne
                          try:
                              colonna_tempo_f.append(float(riga_pulita[0]))
                              colonna_cd.append(float(riga_pulita[2])) # Assumendo Cd sia la 3a colonna (indice 2)
                              colonna_cl.append(float(riga_pulita[3])) # Assumendo Cl sia la 4a colonna (indice 3)
                          except (ValueError, IndexError):
                              print(f"Attenzione: Riga forze non valida o incompleta: {riga.strip()}")
                              continue # Salta la riga problematica
            else: # Usa csv.reader per il tab
                 reader_f = csv.reader(righe_valide_f, delimiter=delimiter_f, skipinitialspace=True)
                 for riga in reader_f:
                      riga_pulita = [val.strip() for val in riga if val.strip()]
                      if len(riga_pulita) >= 4:
                          try:
                              colonna_tempo_f.append(float(riga_pulita[0]))
                              colonna_cd.append(float(riga_pulita[2]))
                              colonna_cl.append(float(riga_pulita[3]))
                          except (ValueError, IndexError):
                              print(f"Attenzione: Riga forze non valida o incompleta: {riga}")
                              continue # Salta la riga problematica

        if colonna_tempo_f:
             force_data_read_success = True
             print(f"Letti {len(colonna_tempo_f)} punti dati Forze.")
        else:
            print("ERRORE: Nessun dato forze valido letto dal file.")
    except Exception as e_f:
        print(f"ERRORE durante la lettura del file forze: {e_f}")
else:
     print("Skipping lettura dati forze perchè file non trovato.")


# --- 2b: Lettura Dati yPlus (se il file è stato trovato) ---
if yplus_data_found:
    print(f"Leggendo dati y+ da {yplus_data_file} per patch '{target_patch}'...")
    try:
        with open(yplus_data_file, 'r') as f_yp:
            righe_valide_yp = [line for line in f_yp if not line.strip().startswith('#')]
            # Il formato yPlus.dat di solito usa spazi come delimitatori
            for riga_yp in righe_valide_yp:
                 riga_pulita_yp = riga_yp.split() # Divide sugli spazi
                 # Controlla se ci sono abbastanza colonne E se la seconda colonna (indice 1) matcha la patch target
                 if len(riga_pulita_yp) >= 5 and riga_pulita_yp[1] == target_patch:
                    try:
                        # Le colonne sono: Time Patch yMin yMax yAvg
                        colonna_tempo_yp.append(float(riga_pulita_yp[0])) # Tempo (indice 0)
                        colonna_yp_min.append(float(riga_pulita_yp[2])) # Min y+ (indice 2)
                        colonna_yp_max.append(float(riga_pulita_yp[3])) # Max y+ (indice 3)
                        colonna_yp_avg.append(float(riga_pulita_yp[4])) # Avg y+ (indice 4)
                    except (ValueError, IndexError) as parse_error_yp:
                         print(f"Attenzione: Riga y+ per patch '{target_patch}' non valida o incompleta: {riga_yp.strip()}")
                         continue # Salta riga problematica

            if colonna_tempo_yp:
                yplus_data_read_success = True # Abbiamo letto dati!
                print(f"Letti {len(colonna_tempo_yp)} punti dati y+ per la patch '{target_patch}'.")
            else:
                print(f"AVVISO: Nessun dato y+ valido trovato per patch '{target_patch}' in {yplus_data_file}. Assicurati che il nome '{target_patch}' sia corretto.")
    except Exception as e_yp:
        print(f"Errore imprevisto durante la lettura del file y+: {e_yp}")
else:
     print("Skipping lettura dati y+ perchè file non trovato.")


# --- Fase 3: Creazione dei Grafici ---
print("Creazione grafici...")

# --- Grafico 1: Forze (Cd e Cl) ---
if force_data_read_success:
    try:
        # Crea la PRIMA figura con 2 subplots per le forze
        fig1, axs1 = plt.subplots(nrows=2, ncols=1, figsize=(12, 8), sharex=True)
        # axs1 è un array numpy, assicurati che sia sempre iterabile anche con 1 solo subplot (non questo caso)
        if not isinstance(axs1, np.ndarray): axs1 = np.array([axs1])
        axs1 = axs1.flatten() # Utile se nrows>1 o ncols>1

        # Plot Cd sul primo subplot
        axs1[0].plot(colonna_tempo_f, colonna_cd, label='Cd', color='blue', linewidth=1.5)
        axs1[0].set_ylabel('Coefficiente di Resistenza (Cd)')
        axs1[0].set_title('Cd vs Tempo')
        axs1[0].legend(); axs1[0].grid(True, linestyle='--', alpha=0.7)

        # Plot Cl sul secondo subplot
        axs1[1].plot(colonna_tempo_f, colonna_cl, label='Cl', color='red', linewidth=1.5)
        axs1[1].set_ylabel('Coefficiente di Portanza (Cl)')
        # axs1[1].set_title('Cl vs Tempo') # Titolo subplot opzionale
        axs1[1].grid(True, linestyle='--', alpha=0.7); axs1[1].legend()

        # Etichetta asse X comune
        axs1[1].set_xlabel('Tempo (s)')
        # Migliora leggibilità asse X
        axs1[-1].xaxis.set_major_locator(MaxNLocator(nbins=15, prune='both')) # Meno tick, evita sovrapposizioni
        plt.setp(axs1[-1].get_xticklabels(), rotation=45, ha="right") # Ruota etichette se necessario


        # Titolo generale per la figura 1
        fig1.suptitle('Coefficienti di Forza vs Tempo', fontsize=16)
        # Aggiusta layout per evitare sovrapposizioni
        plt.figure(fig1.number) # Assicura che le operazioni successive agiscano su fig1
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Lascia spazio per suptitle

        # Salvataggio Grafico 1
        output_filename_f = "Grafico_Forze.png"
        output_path_f = os.path.join(case_path, output_filename_f)
        fig1.savefig(output_path_f, dpi=150)
        print(f"Grafico Forze salvato con successo in: {output_path_f}")
        plt.close(fig1) # IMPORTANTE: Chiudi la figura 1 prima di crearne un'altra

    except Exception as e_plot_f:
        print(f"ERRORE durante la creazione o salvataggio del grafico Forze: {e_plot_f}")
        if 'fig1' in locals() and plt.fignum_exists(fig1.number):
             plt.close(fig1) # Prova a chiudere la figura anche in caso di errore
else:
    print("Dati forze non disponibili o non letti correttamente, grafico Forze non generato.")


# --- Grafico 2: y+ (Se i dati sono stati letti con successo) ---
if yplus_data_read_success:
    print("Creazione grafico y+...")
    try:
        # Crea una NUOVA figura (fig2) con 1 subplot per y+
        fig2, ax2 = plt.subplots(figsize=(12, 6)) # Figura SEPARATA per y+

        ax2.plot(colonna_tempo_yp, colonna_yp_min, label='y+ Min', color='green', linestyle='--', linewidth=1.0)
        ax2.plot(colonna_tempo_yp, colonna_yp_avg, label='y+ Avg', color='black', linestyle='-', linewidth=1.5)
        ax2.plot(colonna_tempo_yp, colonna_yp_max, label='y+ Max', color='purple', linestyle=':', linewidth=1.0)

        ax2.set_ylabel('y+')
        ax2.set_xlabel('Tempo (s)')
        ax2.set_title(f'y+ (Patch: \'{target_patch}\') vs Tempo')
        ax2.grid(True, linestyle='--', alpha=0.7); ax2.legend()
        ax2.xaxis.set_major_locator(MaxNLocator(nbins=15, prune='both')) # Meno tick
        plt.setp(ax2.get_xticklabels(), rotation=45, ha="right") # Ruota etichette
        # ax2.set_yscale('log') # Decommenta se preferisci scala logaritmica per y+

        plt.figure(fig2.number) # Assicura che le operazioni successive agiscano su fig2
        plt.tight_layout() # Aggiusta layout

        # Salvataggio Grafico 2
        output_filename_yp = f"Grafico_yPlus_{target_patch}.png" # Nome file include la patch
        output_path_yp = os.path.join(case_path, output_filename_yp)
        fig2.savefig(output_path_yp, dpi=150)
        print(f"Grafico y+ salvato con successo in: {output_path_yp}")
        plt.close(fig2) # IMPORTANTE: Chiudi la figura 2

    except Exception as e_plot_yp:
        print(f"ERRORE durante la creazione o salvataggio del grafico y+: {e_plot_yp}")
        if 'fig2' in locals() and plt.fignum_exists(fig2.number):
             plt.close(fig2) # Prova a chiudere la figura anche in caso di errore
else:
    # Questo messaggio viene stampato se il file y+ non è stato trovato O se non conteneva dati per la patch specificata
    print("Dati y+ non disponibili o non letti correttamente per la patch specificata, grafico y+ non generato.")


print("--- Script Plotting Completato ---")

--- Inizio Script Plotting OpenFOAM ---
Percorso del caso (directory corrente): /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def
Ricerca directory risultati forze in: /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/forceCoeffs1
Trovato file dati forze in directory temporale: /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/forceCoeffs1/0/forceCoeffs.dat
Ricerca directory risultati y+ in: /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/yPlus1
Trovato file dati y+ in directory temporale: /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/yPlus1/0/yPlus.dat
Leggendo dati forze da: /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/forceCoeffs1/0/forceCoeffs.dat
Letti 1051 punti dati Forze.
Leggendo dati y+ da /home/mattia/OpenFOAM/mattia-12/run/TIROCINIO4def/postProcessing/yPlus1/0/yPlus.dat per patch 'bodyWall'...
Letti 701 punti dati y+ per la patch 'bodyWall'.
Creazione grafici...
Grafico Forze salvato con s